In [12]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [41]:
%run scripts/helper.py
%run scripts/models.py
%run scripts/blending_helper.py

In [42]:
crowd_train = load_file('./data/train.csv/train.csv', index_col='id')
y = y = crowd_train.median_relevance

In [43]:
train_index, test_index = ssSplit(y, train_size=500 ,random_state=1234)

In [44]:
Xtrain = crowd_train.iloc[train_index]
ytrain = y[train_index]

Xtest = crowd_train.iloc[test_index]
ytest = y.iloc[test_index]

In [45]:
Xtrain_text = tweak_text(Xtrain)
Xtest_text = tweak_text(Xtest)

In [46]:
corpus = []

for x in Xtrain_text:
    corpus.append(x)

for x in Xtest_text:
    corpus.append(x)

In [47]:
len(corpus)

1516

In [60]:
from sklearn.cross_validation import StratifiedKFold
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

In [49]:
Xtrain_data, tfv, svd, scl = prepareTrainData(Xtrain_text, corpus)

In [50]:
Xtest_data = prepareTestData(Xtest_text, tfv, svd, scl) 

In [51]:
skf = list(StratifiedKFold(ytrain, 3))

In [56]:
clfs = [SVC(C=10.0, kernel='rbf', gamma=.00, probability=True),
        SVC(C=15.0, kernel='linear', probability=True)
       ]

In [57]:
print "Creating train and test sets for blending."
    
dataset_blend_train = np.zeros((Xtrain_data.shape[0], len(clfs)))
dataset_blend_test = np.zeros((Xtest_data.shape[0], len(clfs)))

Creating train and test sets for blending.


In [58]:
for j, clf in enumerate(clfs):
    print j, clf
    dataset_blend_test_j = np.zeros((Xtest_data.shape[0], len(skf)))
    for i, (train, test) in enumerate(skf):
        print "Fold", i
        X_train = Xtrain_data[train]
        y_train = ytrain[train]
        X_test = Xtrain_data[test]
        y_test = ytrain[test]
        clf.fit(X_train, y_train)
        y_submission = clf.predict_proba(X_test)[:,1]
        dataset_blend_train[test, j] = y_submission
        dataset_blend_test_j[:, i] = clf.predict_proba(Xtest_data)[:,1]
    dataset_blend_test[:,j] = dataset_blend_test_j.mean(1)

0 SVC(C=10.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='rbf', max_iter=-1, probability=True, random_state=None,
  shrinking=True, tol=0.001, verbose=False)
Fold 0
Fold 1
Fold 2
1 SVC(C=15.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='linear', max_iter=-1, probability=True, random_state=None,
  shrinking=True, tol=0.001, verbose=False)
Fold 0
Fold 1
Fold 2


In [68]:
print "Blending."
clf = LogisticRegression()
clf.fit(dataset_blend_train, ytrain)
# y_submission = clf.predict_proba(dataset_blend_test)[:,1]

Blending.


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [71]:
(dataset_blend_train == np.nan).any()

False